In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

import datetime as dt

In [169]:
from ipywidgets import interact
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sova.market_data import generate_market_data
from sova.asset import StockOption, OptionType
from sova.portfolio import Portfolio, Trade

import importlib
import sova
importlib.reload(sova.asset)


@interact(p_val=(0, .1, .0001), q_val=(0, .1, .0001), a0=(0., .1, .0001))
def plot_asset_vol_returns(p_val, q_val, a0):
    print(a0)
    print(p_val)
    print(q_val)

    n_obs = 500
    p_arr = np.array([p_val])
    q_arr = np.array([q_val])
    md = generate_market_data(a0, p_arr, q_arr, n_obs=n_obs)
    x_arr = md.index
    fig1 = go.Figure(
        data=go.Scatter(x=x_arr, y=md.asset_price, line_color='deepskyblue'))
    fig1.update_layout(title='Asset price')
    fig1.add_shape(type="line",
                   x0=md.index[0],
                   y0=md.asset_price[0],
                   x1=md.index[-1],
                   y1=md.asset_price[0],
                   line=dict(
                       color="MediumPurple",
                       width=4,
                       dash="dot",
                   ))
    fig1.show()

    #     fig2 = go.Figure(data=go.Scatter(x=x_arr, y=md.asset_volatility))
    #     fig2.update_layout(title='Volatility')

    opt_call = StockOption(amount=1,
                           expiry=pd.Series(md.index).max(),
                           strike=md.asset_price[0],
                           sigma=md.asset_volatility[0],
                           option_type=OptionType.CALL)
    opt_put = StockOption(amount=1,
                          expiry=pd.Series(md.index).max(),
                          strike=md.asset_price[0],
                          sigma=md.asset_volatility[0],
                          option_type=OptionType.PUT)
    pf = Portfolio(trades=[Trade(x_arr[0], md.asset_price[0], opt_call)])
    #     pf = Portfolio(assets=[call_opt, opt_put])
    pf_price = md.apply(lambda r: pf.current_price(
        stock_price=r.asset_price, market_time=r.name.to_pydatetime()),
                        axis=1)
    pf_delta = md.apply(lambda r: pf.current_delta(
        stock_price=r.asset_price, market_time=r.name.to_pydatetime()),
                        axis=1)

    fig2 = go.Figure(data=go.Scatter(x=x_arr, y=pf_price))
    fig2.update_layout(title='Portfolio PV')
    fig2.show()

    fig3 = go.Figure(data=go.Scatter(x=x_arr, y=pf_delta))
    fig3.update_layout(title='Portfolio Delta')
    fig3.show()

interactive(children=(FloatSlider(value=0.05, description='p_val', max=0.1, step=0.0001), FloatSlider(value=0.…

In [3]:
p_arr = np.array([0.001])
q_arr = np.array([0.002])
md = generate_market_data(0.000001, p_arr, q_arr, n_obs=365)

In [4]:
md.head()

,asset_volatility,asset_return,asset_price,risk_free_rate
market_time,,,,
2020-01-11,0.001001,-0.000008,2.668423,0.01
2020-01-12,0.001001,0.000104,2.668402,0.01
2020-01-13,0.001001,0.001548,2.668680,0.01
2020-01-14,0.001002,-0.000648,2.672815,0.01
2020-01-15,0.001001,-0.001726,2.671083,0.01


In [5]:
for market_date, row in md.iterrows():
    print(market_date)
    print(row)
    break

2020-01-11 00:00:00
asset_volatility    0.001001
asset_return       -0.000008
asset_price         2.668423
risk_free_rate      0.010000
Name: 2020-01-11 00:00:00, dtype: float64


In [6]:
from sova.simulation import HedgeSimulation

In [208]:
a0 = 1e-5
n_obs = 1000
p_arr = np.array([.001])
q_arr = np.array([.002])

delta_threshold = .2

otm_multiplier = 1.01

md = generate_market_data(a0, p_arr, q_arr, n_obs=n_obs)
opt = StockOption(amount=1,
                  expiry=md.index[-1].to_pydatetime(),
                  strike=otm_multiplier * md.asset_price[0],
                  sigma=.1)

portfolio = Portfolio([Trade(curr_mkt_time, curr_mkt.asset_price, opt)], delta_threshold)
sim = HedgeSimulation(portfolio)
sim_df = sim.run_simulation(md)

In [222]:
fig = make_subplots(
    rows=3,
    cols=1,
    subplot_titles=['Asset price', 'Portfolio PV', 'Porfolio Delta'])

fig.append_trace(go.Scatter(x=md.index,
                            y=md.asset_price,
                            line_color='deepskyblue',
                            name='Asset price'),
                 row=1,
                 col=1)

fig.append_trace(go.Scatter(x=[md.index[0], md.index[-1]],
                            y=[opt.strike, opt.strike],
                            line=dict(color='purple', width=4, dash='dash'),
                            name='Strike'),
                 row=1,
                 col=1)

fig.append_trace(go.Scatter(x=md.index,
                            y=sim_df.porfolio_pv,
                            line_color='limegreen',
                            name='Portfolio PV'),
                 row=2,
                 col=1)
fig.append_trace(go.Scatter(x=[md.index[0], md.index[-1]],
                            y=[0, 0],
                            line=dict(color='firebrick', width=2),
                            name='Zero PV'),
                 row=2,
                 col=1)

fig.append_trace(go.Scatter(x=md.index,
                            y=sim_df.portfolio_delta,
                            line_color='mediumpurple',
                            name='Porfolio delta'),
                 row=3,
                 col=1)
fig.append_trace(go.Scatter(x=[md.index[0], md.index[-1]],
                            y=[delta_threshold, delta_threshold],
                            line=dict(color='brown', width=2, dash='dash'),
                            name='Delta threshold'),
                 row=3,
                 col=1)
fig.append_trace(go.Scatter(x=[md.index[0], md.index[-1]],
                            y=[-delta_threshold, -delta_threshold],
                            line=dict(color='brown', width=2, dash='dash'),
                            name='Delta threshold'),
                 row=3,
                 col=1)

fig.update_layout(height=1000, title_text="Delta Hedge Simulation Summary")
fig.show()

In [211]:
sim_df

,asset_price,porfolio_pv,portfolio_delta,hedge_asset_amount,hedge_deposit_amount
2020-01-11,2.434164,0.179858,0.573960,-0.57396,1.397111
2020-01-12,2.430633,0.179751,-0.003523,0.00000,0.000000
2020-01-13,2.430137,0.179641,-0.004074,0.00000,0.000000
2020-01-14,2.426869,0.179547,-0.007355,0.00000,0.000000
2020-01-15,2.443239,0.179445,0.008619,0.00000,0.000000
...,...,...,...,...,...
2022-09-22,2.532987,0.096788,0.011258,0.00000,0.000000
2022-09-23,2.533172,0.096710,0.012872,0.00000,0.000000
2022-09-24,2.527551,0.096565,0.013245,0.00000,0.000000
2022-09-25,2.526002,0.096477,0.013332,0.00000,0.000000
